In [1]:
import numpy as np
from sklearn.datasets import load_digits
import tensorflow as tf
from tensorflow.keras import layers
from tinymlgen import port

In [2]:
def get_data():
    np.random.seed(1337)
    x_values, y_values = load_digits(return_X_y=True)
    x_values /= x_values.max()
    # reshape to (8 x 8 x 1)
    x_values = x_values.reshape((len(x_values), 8, 8, 1))

    # split into train, validation, test
    TRAIN_SPLIT = int(0.6 * len(x_values))
    TEST_SPLIT = int(0.2 * len(x_values) + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

    return x_train, x_test, x_validate, y_train, y_test, y_validate

In [3]:
def get_model():
    x_train, x_test, x_validate, y_train, y_test, y_validate = get_data()

    # create a CNN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(8, 8, 1)))
    # model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(len(np.unique(y_train))))

    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=50, batch_size=16,
                        validation_data=(x_validate, y_validate))
    return model, x_test, y_test

In [4]:
def test_model(model, x_test, y_test):
    x_test = (x_test / x_test.max()).reshape((len(x_test), 8, 8, 1))
    y_pred = model.predict(x_test).argmax(axis=1)

    print('ACCURACY', (y_pred == y_test).sum() / len(y_test))

In [5]:
if __name__ == '__main__':
    model, x_test, y_test = get_model()
    test_model(model, x_test, y_test)

Epoch 1/50


C:\Users\BatuhanYILMAZ\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3082 - loss: 2.1792 - val_accuracy: 0.7333 - val_loss: 1.6376
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8490 - loss: 1.3862 - val_accuracy: 0.8444 - val_loss: 0.9010
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9330 - loss: 0.6590 - val_accuracy: 0.8417 - val_loss: 0.6212
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9407 - loss: 0.4142 - val_accuracy: 0.8583 - val_loss: 0.5175
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9368 - loss: 0.2775 - val_accuracy: 0.8500 - val_loss: 0.4776
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9531 - loss: 0.2159 - val_accuracy: 0.8611 - val_loss: 0.4646
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9569 - loss: 0.1915 - val_accuracy: 0.8583 - val_loss: 0.4424
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9690 - loss: 0.1598 - val_accuracy: 0.8667 - val_loss: 0.4244
Epo

In [10]:
model.save('my_model.keras')

In [35]:
# convert and save the model in TFLite format

!mkdir output
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("output/cnn_mnist_model.tflite", "wb").write(tflite_model)

mkdir: cannot create directory ‘output’: File exists


13820

In [36]:
!xxd -i output/cnn_mnist_model.tflite > output/cnn_mnist_model.h

In [38]:
# TO GET THE ORIGINAL INPUTS, RECALL THE FUNCTIONS FROM ABOVE

# Load and preprocess the digits dataset
def get_data():
    np.random.seed(1337)
    x_values, y_values = load_digits(return_X_y=True)
    x_values /= x_values.max()
    # reshape to (8, 8, 1)
    x_values = x_values.reshape((len(x_values), 8, 8, 1))
    return x_values, y_values

# Get the data
x_values, y_values = get_data()

# Function to print the input data in Arduino-friendly format
def print_arduino_format(data, label):
    flattened_data = data.flatten()
    print(f"// Digit {label}")
    print("float x_test[64] = {")
    for i in range(len(flattened_data)):
        if i % 8 == 0 and i != 0:  # Print 8 values per line for readability
            print()
        print(f"{flattened_data[i]:.6f}f,", end=" ")
    print("\n};\n")

# Create and print one sample input for each digit (0-9)
for digit in range(10):
    # Find the first occurrence of the digit in the dataset
    index = np.where(y_values == digit)[0][0]
    sample_input = x_values[index]
    print_arduino_format(sample_input, digit)

// Digit 0
float x_test[64] = {
0.000000f, 0.000000f, 0.312500f, 0.812500f, 0.562500f, 0.062500f, 0.000000f, 0.000000f, 
0.000000f, 0.000000f, 0.812500f, 0.937500f, 0.625000f, 0.937500f, 0.312500f, 0.000000f, 
0.000000f, 0.187500f, 0.937500f, 0.125000f, 0.000000f, 0.687500f, 0.500000f, 0.000000f, 
0.000000f, 0.250000f, 0.750000f, 0.000000f, 0.000000f, 0.500000f, 0.500000f, 0.000000f, 
0.000000f, 0.312500f, 0.500000f, 0.000000f, 0.000000f, 0.562500f, 0.500000f, 0.000000f, 
0.000000f, 0.250000f, 0.687500f, 0.000000f, 0.062500f, 0.750000f, 0.437500f, 0.000000f, 
0.000000f, 0.125000f, 0.875000f, 0.312500f, 0.625000f, 0.750000f, 0.000000f, 0.000000f, 
0.000000f, 0.000000f, 0.375000f, 0.812500f, 0.625000f, 0.000000f, 0.000000f, 0.000000f, 
};

// Digit 1
float x_test[64] = {
0.000000f, 0.000000f, 0.000000f, 0.750000f, 0.812500f, 0.312500f, 0.000000f, 0.000000f, 
0.000000f, 0.000000f, 0.000000f, 0.687500f, 1.000000f, 0.562500f, 0.000000f, 0.000000f, 
0.000000f, 0.000000f, 0.187500f, 0.937500f

In [21]:
interpreter = tf.lite.Interpreter(model_path='cnn_mnist_model_quantized.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

correct_predictions = 0
for i in range(len(x_test)):
    input_data = np.expand_dims(x_test[i], axis=0).astype(np.float32)  # Ensure input data is float32
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_index).argmax()
    if prediction == y_test[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(x_test)
print('Quantized model accuracy:', accuracy)

Quantized model accuracy: 0.9498607242339833
